# Dekompozycja szeregu

W trakcie budowy modelu przeprowadza się **dekompozycje szeregu czasowego** w zależności od przyjętych założeń.

**Oznaczenia**: $m_t$ - trend, $s_t$ - sezonowość, $\varepsilon_t$ - szum.

**Model addytywny** - wielkość wahań sezonowych lub wariancja danych wokół tendencji długoterminowej (trendu) nie zmienia się wraz z poziomem szeregu:
$$y_t = m_t + s_t + \varepsilon_t$$

**Model multiplikatywny** - amplituda wahań sezonowych lub wariancja danych proporcjonalna do poziomu szeregu:
$$y_t = m_t \cdot s_t \cdot \varepsilon_t$$


In [ ]:
import numpy as np
import pandas as pd

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

#from google.colab import files
#src = list(files.upload().values())[0]
#open('ts_utils.py','wb').write(src)

#from ts_utils import check_time_series_stationary

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from statsmodels.tsa.stattools import adfuller
def check_time_series_stationary(y, rolling_len = 12):
    
    y = pd.Series(y)
    
    df_test = adfuller(y)
    
    if df_test[1] < 0.05:
        print('Szereg jest stacjonarny')
    else:
        print('Szereg jest niestacjonarny')
    
    print("{0:^32s}".format("Dickey-Fuller Test"))
    print("-"*32+"\n")
    
    print("{0:<20s}: {1:>10.4f}".format('Test Statistic', df_test[0]))
    print("{0:<20s}: {1:>10.4f}".format('p-value', df_test[1]))
    print("-"*32+"\n")
    
    rolling_mean = y.rolling(rolling_len).mean()
    rolling_var = y.rolling(rolling_len).var()

    plt.plot(y)
    plt.plot(rolling_mean, label="Rolling mean")
    plt.plot(rolling_var, label="Rolling var")
    plt.legend()
    plt.show()
    
    print("{0:^32s}".format("Autocorrelation plot"))
    print("-"*32+"\n")
    pd.plotting.autocorrelation_plot(y)
    plt.show()

## Algorytm dekompozycji szeregu czasowego

1. Wyznaczamy oszacowany składnik trendu $\hat{m_t}$, wykorzystując np. metodę średniej ruchomej.
2. Oszacowany trend jest eliminowany z danych. Wyznaczamy szereg:

    a) dekompozycja addytywna: $y_t - \hat{m_t}$,
    
    b) dekompozycja multiplikatywna: $y_t/\hat{m_t}$.
    
3. Wyznaczamy indeksy sezonowe dla poszczególnych miesięcy, kwartałów, itd. Indeksy sezonowe są wyznaczane poprzez uśrednienie wartości szeregu z kroku 2 dla każdej jednostki czasu (np. miesiąc, kwartał) i wszystkich okresów (lat).
4. Standaryzujemy wskaźniki sezonowe, tak aby nie miały one wpływu na tendencję długoterminową.
5. Wyznaczamy reszty, usuwając trend i sezonowość

    a) dekompozycja addytywna: $\varepsilon_t = y_t - \hat{m_t} - \hat{s_t}$,
    
    b) dekompozycja multiplikatywna: $\varepsilon_t = y_t/\hat{m_t}/\hat{s_t}$.


## ***Zadanie 1

a) Przy użyciu metody średniej ruchomej oszacuj trend dla szeregu `southern`, wiedząc, że dla szeregu długości $n$, tzn. $Y_1, \ldots, Y_n$ estymacja średniej ruchomej w chwili $t$ jest postaci:
$$m(t)=\frac{1}{2q+1}\sum_{i=-q}^qY_{t-i},$$
gdzie $q$ jest rzędem średniej ruchomej (np. `convolution_filter` z `statsmodels.tsa.filters.filtertools`). 

Innymi słowy, aby wyznaczyć oszacowaną wartość trendu w chwili $t$, uśredniamy $q$ poprzedzających ten moment wartości i $q$ wartości następujących po tym momencie.

b) Następnie wyznacz indeksy sezonowe dla mięsięcy a następnie wystandaryzuj je przez średnią.

c) Oszacuj wartość reszt szeregu, sprawdź jego stacjonarność.

Załóż, że szereg `southern` jest szeregiem addytywnym.

In [ ]:
# #chcemy dekomponować szereg, ponieważ łatwiej jest modelować poszczególne części (np. trend regrsją liniową czy wielomianową)
# #sezonowość, przez powtarzanie dla konkretnych miesięcy, średnich wyliczonych na danych miesiącach na części uczącej pozbawionej trendu
# #losowość (jeśli stacjonarna to np. ARMA)
# southern = pd.read_csv('southern.csv', 
#                  parse_dates=['Date'], index_col='Date')

In [ ]:
# southern.plot()

#### Oszacowanie trendu

In [ ]:
# from statsmodels.tsa.filters.filtertools import convolution_filter
# # używamy metody convolution_filter, jeśli n_sizes = 2 sumujemy wszystkie elmenty odległe o freq/2 z dwóch stron od zadanego punktu i tak przechodzimy punkt po punkcie
# # innymi słowy robimy sumę ruchomą

In [ ]:
# #średnia ruchoma
# period = 12
# trend = convolution_filter(southern.Southern, np.ones(period + 1), nsides=2) / (period + 1)

In [ ]:
# #wykres średniej ruchomej
# plt.plot(southern)
# plt.plot(trend)

#### Oszacowanie sezonowości

In [ ]:
# #wyrzucamy trend
# detrended = southern.Southern - trend
# plt.plot(detrended)

In [ ]:
# #liczymy na szeregu bez trendu średnie dla poszczególnych miesięcy
# month_avgs = np.array([np.nanmean(detrended[i::period], axis=0) for i in range(period)])
# print(month_avgs)
# month_avgs -= np.mean(month_avgs, axis=0)
# print(np.mean(month_avgs, axis=0))
# #powtarzamy średnie tyle razy ile mamy lat, w efekcie mamy szereg długości southern, 
# #gdzie dla każdego miesiąca w każdym roku, mamy średnią wyliczoną dla danego miesiąca 
# seasonal = np.tile(month_avgs.T, southern.size // period).T
# seasonal = pd.Series(seasonal, index=southern.index)

In [ ]:
# plt.plot(seasonal)

In [ ]:
# #rozpatrujemy część losową - bez średniej i bez trendu
# resid = detrended - seasonal
# plt.plot(resid)

In [ ]:
# plt.plot(southern)
# plt.show()
# plt.plot(trend)
# plt.show()
# plt.plot(seasonal)
# plt.show()

#### Oszacowanie elementu losowego i diagnostyka stacjonarności

In [ ]:
# check_time_series_stationary(resid.dropna())

## Zadanie 2
Dokonaj dekompozycji szeregu przy użyciu `seasonal_decompose` dla następujących danych:
- `southern`,
- `a10`,
- `AirPassengers`, 
- `sunspots`.

### `southern`

In [ ]:
# southern = pd.read_csv('southern.csv', 
#                  parse_dates=['Date'], index_col='Date')


In [ ]:
# southern_dec = seasonal_decompose(southern,model = 'additive')

In [ ]:
# southern_dec.plot()
# plt.show()

In [ ]:
# southern_dec.trend
# southern_dec.seasonal
# southern_dec.resid[0:7]

### `a10`

In [ ]:
# a10 = pd.read_csv('a10.csv', parse_dates=['date'], index_col='date')
# a10.head()

In [ ]:
# a10.plot()

In [ ]:
# a10_dec = seasonal_decompose(a10,model = 'multiplicative')
# a10_dec.plot()
# plt.show()

In [ ]:
# check_time_series_stationary(a10_dec.resid.dropna())

### `AirPassengers`

In [ ]:
# air_passengers = pd.read_csv('AirPassengers.csv', 
#                  parse_dates=['Month'], index_col='Month')

# air_passengers.head()

### `sunspot` 
Time Series:
Start = 1700 
End = 1988 
Frequency = 1 

In [ ]:
# sunspot = pd.read_csv('sunspot.csv')
# sunspot

In [ ]:
# sunspot.index = pd.date_range('1700', '1988', freq='AS') #nadajemy rok jako indeks dla poszczególnych obserwacji
# sunspot.index

In [ ]:
# sunspot.plot()

In [ ]:
# sunspot_dec = seasonal_decompose(sunspot,model = 'additive')
# sunspot_dec.plot()
# plt.show()

In [ ]:
# plt.plot(sunspot[(sunspot.index >= '1700') & (sunspot.index <= '1760')])
# plt.show()

In [ ]:
# sunspot_dec = seasonal_decompose(sunspot,model = 'additive',freq = 10)
# sunspot_dec.plot()
# plt.show()

In [ ]:
# check_time_series_stationary(sunspot_dec.resid.dropna()) 

In [ ]:
# plt.plot(sunspot_dec.resid.dropna())